In [138]:
# Maths Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Jupyter Imports
from IPython.core.display import display, HTML, Math
from ipywidgets import interact, fixed, FloatSlider

# We like tables
import tabulate

## Graph Config (borrowed from StatsForLab)
%matplotlib inline
# Peter Sloan is very particular about his graphs.
# These next lines will help Peter-i-fy your graph. Again, don't worry too much about how this works for now.
from matplotlib import rcParams
rcParams['xtick.direction'] = 'in'
rcParams['ytick.direction'] = 'in'
rcParams['errorbar.capsize'] = 3
rcParams['lines.markersize'] = 10
rcParams['figure.figsize'] = 8, 5


# Introduction to Modelling

I don't really know what these documents are supposed to be. This is based off of @Samuel Cottee's SHM python files — with some additions by me (@Joshua Coles) to make it pretty in a Jupyter notebook.

At the moment this is designed as a showcase of what Jupyter notebooks can do / possibly the start of a "handout" to be given to students to play around with and act as instructions for an assignment and an example of the *output* that is to be expected from running the different tasks.

This document uses python concepts which are probably to complex to be shown to students directly, but the code itself can be hidden providing snippets to guide them.

```py
def acc(x, k, m):
    return -(k / m) * x
```

## Model

First we make a simple SHM model, the system is constructed as `SHM(k, m, initial_x, initial_v)` which can
either be subjected to a time step model or, as the theory is relatively simple, the "ideal" graphs shown.

In [139]:
class SHM(object):
    def __init__(self, k, m, initial_x, initial_v):
        self.k = k
        self.m = m
        self.initial_x = initial_x
        self.initial_v = initial_v

        # Theory variables
        self.omega = np.sqrt(k/m)
        self.phi = np.arctan2(initial_v, -self.omega * initial_x)
        self.amplitude = (initial_x / np.cos(self.phi)) or \
                         (initial_v / (-self.omega * np.sin(self.phi)))


    def theory(self, t):
        x = self.amplitude * np.cos(self.omega * t + self.phi)
        v = -self.amplitude * self.omega * np.sin(self.omega * t + self.phi)
        return [x, v]


    def acc(self, x):
        return -(self.k / self.m) * x


    def model(self, dt, t_max):
        time_steps = np.arange(0, t_max, dt)
        n = len(time_steps)

        # Initial Conditions
        x = [self.initial_x]
        v = [self.initial_v]

        for i in range(n - 1):
            v.append(v[i] + self.acc(x[i]) * dt)
            x.append(x[i] + v[i] * dt)

        return [time_steps, x, v]

    # This function is provided for scipy.optimize.curve_fit
    # It must be in the form f(independent variable, parameters)
    @staticmethod
    def curve_fit_x(t, amplitude, omega, phi):
        return amplitude * np.cos(omega * t + phi)

## SHM Playground

Here we provide a plot of the `x, v` variables as time progresses, with sliders to help the students understand
what changing each thing does, this might be more helpful for example, in DHM.

In [140]:
k_slider = FloatSlider(min=0.1, max=5, step=0.1)
m_slider = FloatSlider(min=0.1, max=5, step=0.1)

initial_x_slider = FloatSlider(min=-3, max=3, step=0.1)
initial_v_slider = FloatSlider(min=-3, max=3, step=0.1)

@interact(k=k_slider, m=m_slider, initial_x=initial_x_slider, initial_v=initial_v_slider)
def shm_playground(k, m, initial_x, initial_v):
    shm = SHM(k, m, initial_x, initial_v)

    t = np.arange(0, 10, 0.01)
    x, v = shm.theory(t)

    plt.figure()

    plt.plot(t, x, label="Position")
    plt.plot(t, v, label="Velocity")

    plt.legend()
    plt.show()

    # Energies

    ke = 0.5 * m * v ** 2
    pot = 0.5 * k * x ** 2

    plt.figure()

    plt.plot(t, ke, label="Kinetic Energy")
    plt.plot(t, pot, label="Potential Energy")
    plt.plot(t, ke + pot, label="Total Energy")

    plt.legend()
    plt.show()


interactive(children=(FloatSlider(value=0.1, description='k', max=5.0, min=0.1), FloatSlider(value=0.1, descri…

## Time Step Modelling

Now we move onto applying the time step modelling to the system, here we use the `k, m, x0, v0` from the
previous examples and provide new sliders for `dt, t_max`

In [141]:

n_k_slider = FloatSlider(min=0.1, max=5, step=0.1)
n_m_slider = FloatSlider(min=0.1, max=5, step=0.1)

n_initial_x_slider = FloatSlider(min=-3, max=3, step=0.1)
n_initial_v_slider = FloatSlider(min=-3, max=3, step=0.1)

@interact(k=n_k_slider, m=n_m_slider, initial_x=n_initial_x_slider, initial_v=n_initial_v_slider, dt=(0.01, 2, 0.01), t_max=(0, 10, 1))
def timestep_playground(k, m, initial_x, initial_v, dt, t_max):
    shm = SHM(k, m, initial_x, initial_v)

    time_steps, x, v = shm.model(dt, t_max)

    plt.figure()

    plt.plot(time_steps, x, color="maroon", label="Model Position")
    plt.plot(time_steps, v, color="olive", label="Model Velocity")

    smooth_time = np.arange(0, t_max, 0.01)
    theory_x, theory_v = shm.theory(smooth_time)

    plt.plot(smooth_time, theory_x, linestyle='--', color="maroon", label="Theoretical Position")
    plt.plot(smooth_time, theory_v, linestyle='--', color="olive", label="Theoretical Velocity")

    plt.legend()
    plt.show()

    # Now we can use the scipy curve_fit function to test how the simulation matches up with prediction.
    popt, pcov = curve_fit(
        f = SHM.curve_fit_x,

        xdata = time_steps,
        ydata = x,

        p0=[np.max(x), shm.omega, 0]
    )

    cf_amplitude, cf_omega, cf_phi = popt

    table = [
        ["", "Amplitude", "Omega", "Phi"],
        ["Theoretical", shm.amplitude, shm.omega, shm.phi],
        ["Fitted", cf_amplitude, cf_omega, cf_phi]
    ]

    display(HTML(tabulate.tabulate(table, tablefmt='html')))

    plt.figure()

    plt.plot(smooth_time, theory_x, linestyle='--', label="Theoretical Position")
    plt.plot(smooth_time, SHM.curve_fit_x(smooth_time, cf_amplitude, cf_omega, cf_phi), label="Fitted Position")

    plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=0.1, description='k', max=5.0, min=0.1), FloatSlider(value=0.1, descri…

Possible comments to make

- Tell them to play with the sliders
- Comment on accuracy:
    - More accurate as `dt` decreases
    - Less accuracy as `t` increases.
